#### 1. Example Code to Create a composed rand cuboid group

In [ ]:
import time, sys
import numpy as np
sys.path.append('..')
from pb_diff_envs.environment.static.comp_kuka_env import ComposedDynKukaEnv
from pb_diff_envs.environment.static.comp_cuboid_group import ComposedRandCuboidGrp

num_groups = 5
num_walls_c = np.array([4, 2], dtype=np.int32)
hr = 0.20 # 0.2
vr_x = 0.2; vr_y = 0.2;vr_z = (0., 0.)
se_x = 0.5; se_y = 0.5; se_z = (0.3, 0.9) # 0.4 too difficult
hExt_range = [hr, hr, hr] # fixed-size obstacle
seed = 333

## check z
start_end = np.array([[-se_x, se_x], [-se_y, se_y], se_z])
half_extents_c = np.array(hExt_range)[None,].repeat(2, axis=0)
void_range = np.array( [(-vr_x, vr_x), (-vr_y, vr_y), vr_z] ) # (0., 0.)


cg = ComposedRandCuboidGrp('luotest-comp-kuka-vgrp-ipynb', num_groups, 2, num_walls_c, start_end, void_range, 
                            half_extents_c, seed, gen_data=True, 
                            robot_class=ComposedDynKukaEnv.robot_class,
                            is_eval=True, debug_mode=False, GUI=False, grp_seed_diff=True, rand_iter_limit=1e5)


#### 2. Example Code to Create a comp vgrp

In [ ]:
import time, sys
sys.path.append('..')
from pb_diff_envs.environment.static.comp_kuka_env import ComposedDynKukaEnv

import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import pybullet as p
import pb_diff_envs.objects.static.voxel_group as vv; reload(vv)
from pb_diff_envs.environment.static.kuka_rand_vgrp import Kuka_VoxelRandGroupList
no_while = True

mazelist_config = dict(planner_timeout=30, 
                        interp_density=3,
                       min_episode_distance=4*np.pi)

import gym
env_name = 'Compkuka7d-testOnly-rand-v11'
# env_name = 'Compkuka7d-rand-nv4nv3-se0505-vr0202-hExt20-v0'
dvg_group = gym.make(env_name, load_mazeEnv=False, gen_data=True, is_eval=True) # True



In [ ]:
if p.isConnected():
    p.disconnect()
p.isConnected()

## simple vis of env

In [ ]:
if p.isConnected():
    p.disconnect()
env = dvg_group.create_single_env(0) # 
env.unload_env()
env.load(GUI=False) # important not delete
p.getContactPoints()
p.stepSimulation()
# env.robot.get_workspace_observation()
plt.imshow(env.render())
plt.show()
# p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)  # Disable the default GUI controls
# p.configureDebugVisualizer(p.COV_ENABLE_MOUSE_PICKING, 1)  # Enable mouse picking for camera control
p.addUserDebugLine([0, 0, 0], [1, 0, 0], lineColorRGB=[1, 0, 0], lineWidth=6)
p.addUserDebugLine([0, 0, 0], [0, 1, 0], lineColorRGB=[0, 1, 0], lineWidth=6)
p.addUserDebugLine([0, 0, 0], [0, 0, 1], lineColorRGB=[0, 0, 1], lineWidth=6)
while True:
    p.stepSimulation()
    # p.performCollisionDetection()
    c_pts = p.getContactPoints()
    # print(p.get)
    time.sleep(0.1)
    if c_pts is not None and len(c_pts) > 0:
        # print(p.getContactPoints())
        pass
        # time.sleep(2)
        # p.resetJointStatesMultiDof(env.robot_id, np.arange(7), [[0.]]*7)
    if no_while:
        break

## Sample a problem/episode

In [ ]:
from utils.utils import DotDict
from tqdm import tqdm
env.load(GUI=False) # False
prev_pose = np.array([0.0] * 7)
env.robot.set_config(prev_pose)
solutions = []
# n_traj = 25
n_traj = 5 # 20
for i in tqdm(range(n_traj)):
    solution, _ = env.sample_1_episode(prev_pose)
    solution = np.array(solution)
    prev_pose = solution[-1]
    solutions.append(solution)



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data
import numpy as np
from utils.utils import save_gif
from IPython.display import HTML
import base64
from objects.trajectory import WaypointLinearTrajectory, WaypointProportionTrajectory
from utils.kuka_utils_luo import SolutionInterp
# Visualization
from importlib import reload
import utils.robogroup_utils_luo  as rul; reload(rul)
from utils.robogroup_utils_luo import robogroup_visualize_traj_luo
from utils.kuka_utils_luo import visualize_kuka_traj_luo

sol_interp = SolutionInterp(density=mazelist_config['interp_density'])
sol_i = solutions[0] # which motion trajectory to visualize
result = DotDict(solution=sol_i)
traj_vis = sol_interp(result.solution,)


gifs, ds, vis_dict = visualize_kuka_traj_luo(env, traj_vis, is_debug=False)


gif_name = f'./test_comp_kuka.gif'
save_gif(gifs, gif_name, duration=ds)
b64 = base64.b64encode(open(gif_name, 'rb').read()).decode('ascii')
display(HTML(f'<img src="data:image/gif;base64,{b64}" />'))

print(traj_vis.shape)
print(result.solution.shape)

In [ ]:
# while True:
#     time.sleep(0.1)
#     p.stepSimulation()